In [ ]:
import io, os, operator, bisect
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class ST:
  def __init__(self, seg, f, default=0):
    self._def = default
    self._f = f
    self._len = len(seg)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self.seg = [default] * (2 * _size)
    self.seg[_size:_size + self._len] = seg
    for i in reversed(range(_size)):
      self.seg[i] = f(self.seg[i + i], self.seg[i + i + 1])

  def __getitem__(self, i):
    return self.seg[i + self._size]

  def __setitem__(self, i, v):
    i += self._size
    self.seg[i] = v
    i //= 2
    while i:
      self.seg[i] = self._f(self.seg[2 * i], self.seg[2 * i + 1])
      i //= 2

  def query(self, s, e):
    s += self._size
    e += self._size

    l = r = self._def
    while s < e: 
      if s & 1:
        l = self._f(l, self.seg[s])
        s += 1
      if e & 1:
        e -= 1
        r = self._f(self.seg[e], r)
      s //= 2
      e //= 2

    return self._f(l, r)

MAX = 100000
def sol() :
  N = int(input())
  T = list(map(int, input().split()))
  Tzip = {v:i for i, v in enumerate(T)}
  B = list(map(int, input().split()))
  C = list(map(int, input().split()))

  st = ST([0] * (MAX + 1), operator.add) # 시각 N에서 얻을 수 있는 최대의 혼란
  for i, (t, b, c) in enumerate(zip(T, B, C)) :
    res = st.query(bisect.bisect_left(T, t-b), i) #해당 시간에서 얻을 수 있는 혼란의 증가량
    if res > c : continue
    st[i] = c - res
  print(st.seg[1])

sol()

### 자력솔
- 뭔가 그리디나 DP인거 같은데, 자꾸만 구간합을 구해야 한다는 생각에 지배당해 세그먼트 트리로 푼 문제.
- IndexError가 나서 뭔가 싶었는데 좌표압축이 필요한것 같아 보였다. 다만, 이미 T는 strictly increasing 하므로 이분탐색만 진행해도 됐다. 필요한 쿼리는 `bisect_left()`